In [ ]:
'''import page''' 
import scipy.io as scio
import numpy as np
from collections import defaultdict
import pickle

In [ ]:
'''
数据集读取，mat文件里的格式如下

rating.mat

rating.mat includes the rating information. there are five columns and they are userid, productid, categoryid, rating, helpfulness, respectively. 

*****************************************************************
For example, for one row
(1,2,3,4,5)

It means that user 1 gives a rating of 4 to the product 2 from the category 3. The helpfulness of this rating is 5. 

*****************************************************************************


===============================================================================================
trustnetwork.mat

trustnetwork.mat includes the trust relations between users. There are two columns and both of them are userid.

*************************************************************************************
for example, for one row,
(1,2)

it means that user 1 trusts user 2.
*************************************************************************************


'''
path = 'data/ciao'#数据集的相对路径
trust = scio.loadmat(path + '/trustnetwork.mat')
trust = trust['trustnetwork']
print(trust)#社交网络
rating = scio.loadmat(path + '/rating.mat')
rating = rating['rating']
print(rating)#用户-物品交互网络

In [ ]:
'''去掉没有社会关系的用户'''
delete = []
print(len(trust))
for i in range(len(trust)):
    if trust[i,0] == trust[i,1]:
        print(i)
        delete.append(i)
        print(i)
trust = np.delete(trust,delete,0)

print(len(trust))
'''
numpy.delete(arr, obj, axis)

arr：需要处理的矩阵
obj：在什么位置处理
axis：这是一个可选参数，axis = None，1，0

axis=None：arr会先按行展开，然后按照obj，删除第obj-1（从0开始）位置的数，返回一个行矩阵。
axis = 0：arr按行删除
axis = 1：arr按列删除
所有操作都是在arr的副本进行，需要有变量接收返回值。

'''

In [ ]:
'''用户,评分，物品初始矩阵构建'''
dic_user_o2i = {}
i = 0
# build user index
for user in trust.reshape(-1,):
  if user in dic_user_o2i.keys():
    continue
  else:
    dic_user_o2i[user] = i
    i += 1
print('building from trust',len(dic_user_o2i))

delete = []
for i in range(len(rating)):
  if rating[i,0] not in dic_user_o2i.keys():
    delete.append(i)
rating = np.delete(rating, delete, 0)
print('rating_num:',len(rating))

dic_item_o2i = {}
i = 0
for item in rating[:, 1]:
  if item in dic_item_o2i.keys():
    continue
  else:
    dic_item_o2i[item] = i
    i += 1
print('number of items ', len(dic_item_o2i.keys()))

In [ ]:
history_u_lists = defaultdict()
history_ur_lists = defaultdict()
history_v_lists = defaultdict()
history_vr_lists = defaultdict()
social_adj_lists = defaultdict()
item_adj_lists = defaultdict()
ratings_list = defaultdict()


    

data_total = []
print('building social_adj_lists')
for user in range(len(dic_user_o2i)):
  social_adj_lists[user] = []
for line in trust:
  social_adj_lists[dic_user_o2i[line[0]]].append(dic_user_o2i[line[1]])
  social_adj_lists[dic_user_o2i[line[1]]].append(dic_user_o2i[line[0]])
#print(len(dic_user_o2i)) #7317个用户
for user in range(len(dic_user_o2i)):
  social_adj_lists[user] = set(social_adj_lists[user])

print(social_adj_lists[7316])

print('building ratings_list')
i = 0
for rate in set(rating[:, 3]):
  ratings_list[rate] = i
  i += 1
  
print('building other dicts')
for user in range(len(dic_user_o2i)):
  history_u_lists[user] = []
  history_ur_lists[user] = []
for item in range(len(dic_item_o2i)):
  history_v_lists[item] = []
  history_vr_lists[item] = []
  
rating = rating[:, [0, 1, 3]]
np.random.shuffle(rating)
valid = rating[ : int(0.2 * len(rating))]
test = rating[int(0.2 * len(rating)) : int(0.4 * len(rating))]
train = rating[int(0.4 * len(rating)): ]

In [ ]:
def build_item_adj_lists(history_v_lists):
    adj_lists = {}
    for key in history_v_lists.keys():
        adj_lists[key] = []
    for key in history_v_lists.keys():
        for key_temp in history_v_lists.keys():
            if key != key_temp and len(set(history_v_lists[key]) | set(history_v_lists[key_temp])) != 0:
                if len(set(history_v_lists[key]) & set(history_v_lists[key_temp])) / len(set(history_v_lists[key]) | set(history_v_lists[key_temp])) > 0.5:
                    adj_lists[key].append(key_temp)
                    #print("添加item之间的连接")
    return adj_lists

print('validset')
validset = []
testset = []
for line in valid:
  user = line[0]
  item = line[1]
  rate = ratings_list[line[2]]
  if user in dic_user_o2i.keys():
    validset.append([dic_user_o2i[user], dic_item_o2i[item], rate])  
print('testset')
for line in test:
  user = line[0]
  item = line[1]
  rate = ratings_list[line[2]]
  if user in dic_user_o2i.keys():
    testset.append([dic_user_o2i[user], dic_item_o2i[item], rate]) 
print('trainingset')
for line in train:
  user = line[0]
  item = line[1]
  rate = ratings_list[line[2]]
  if user in dic_user_o2i.keys():
    history_u_lists[dic_user_o2i[user]].append(dic_item_o2i[item])
    history_ur_lists[dic_user_o2i[user]].append(rate)
    history_v_lists[dic_item_o2i[item]].append(dic_user_o2i[user])
    history_vr_lists[dic_item_o2i[item]].append(rate)
    data_total.append([dic_user_o2i[user], dic_item_o2i[item], rate])

print("line读取完成")
item_adj_lists = build_item_adj_lists(history_v_lists)
print("共享50%的user的item连接完成")

pickle_data = [history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, data_total, validset, testset, social_adj_lists, item_adj_lists, ratings_list]

#different pkl
with open("data/ciao000.pkl", 'wb') as fo:
    pickle.dump(pickle_data, fo)